In [ ]:
print('Total number of possible outputs: \n', 2**256)

Total number of possible outputs: 
 115792089237316195423570985008687907853269984665640564039457584007913129639936


In [ ]:
import hashlib
h = hashlib.sha256()
h.update('SHANKARAM!'.encode('utf-8'))

print(h.hexdigest())
print(int(h.hexdigest(), 16))

164ac7284d560dc73549acef99935a9081dfce63c95d20e3d9f96727eca6b796
10083003885690785939927329789380586011025278465845892625957090961538846996374


In [ ]:
int(h.hexdigest(), 16) < 2**256

True

In [ ]:
h1 = hashlib.sha256()
h1.update('SHANKARAM!'.encode('utf-8'))
print("Hash1: ", int(h1.hexdigest(), 16))

h2 = hashlib.sha256()
h2.update('SHANKARA!'.encode('utf-8'))
print("Hash2: ", int(h2.hexdigest(), 16))

Hash1:  10083003885690785939927329789380586011025278465845892625957090961538846996374
Hash2:  43185941668748673188092787428966825748993548073089542176970726640722655523542


In [ ]:
import hashlib

class Block():
    def __init__(self, data, previous_hash):
        self.hash = hashlib.sha256()
        self.previous_hash = previous_hash
        self.nonce = 0
        self.data = data
        
    def mine(self, difficulty):
        self.hash.update(str(self).encode('utf-8'))
        while int(self.hash.hexdigest(), 16) > 2**(256-difficulty):
            self.nonce += 1
            self.hash = hashlib.sha256()
            self.hash.update(str(self).encode('utf-8'))
        
    def __str__(self):
        return "{}{}{}".format(self.previous_hash.hexdigest(), self.data, self.nonce)

In [ ]:
class Block():
    def __str__(self):
        return "Hello World!" 

blck_obj = Block()
print(blck_obj) 

Hello World!


In [ ]:
import hashlib

class Chain():
    def __init__(self, difficulty):
        self.difficulty = difficulty
        self.blocks = []
        self.pool = []
        self.create_origin_block()

    def proof_of_work(self, block):
        hash = hashlib.sha256()
        hash.update(str(block).encode('utf-8'))
        return block.hash.hexdigest() == hash.hexdigest() and\
                                        int(hash.hexdigest(), 16) < 2**(256-self.difficulty) and\
                                        block.previous_hash == self.blocks[-1].hash
        
    def add_to_chain(self, block):
        if self.proof_of_work(block):
            self.blocks.append(block)
            
    def add_to_pool(self, data):
        self.pool.append(data)
        
    def create_origin_block(self):
        h = hashlib.sha256()
        h.update(''.encode('utf-8'))
        origin = Block("Origin", h)
        origin.mine(self.difficulty)
        self.blocks.append(origin)
        
    def mine(self):
        if len(self.pool) > 0:
            data = self.pool.pop()
            block = Block(data, self.blocks[-1].hash)
            block.mine(self.difficulty)
            self.add_to_chain(block)
            print("\n=========================================================")
            print("Previous_hash: ", block.previous_hash.hexdigest())
            print("Block_data: ", block.data)
            print("Hash: ", block.hash.hexdigest())
            print("Nonce: ", block.nonce)
            print("==========================================================\n")


In [ ]:
# Setting difficulty at the chain level to '20' 
chain = Chain(20)

i = 0

while(True):
    data = input("Add something to the chain: ")
    chain.add_to_pool(data)
    chain.mine()
    # print(chain.blocks[i])
    i += 1

Add something to the chain: Shankaram

Previous_hash:  00000dcfe33cdc83865d5a2e02158288eb65e8f688e511634882d43f2e49ad1f
Block_data:  Shankaram
Hash:  000008d5e35c98a4a246d3ffb35814cb6619d5ac9d145035d5f81b7db2e5bae4
Nonce:  214828

Add something to the chain: Siva

Previous_hash:  000008d5e35c98a4a246d3ffb35814cb6619d5ac9d145035d5f81b7db2e5bae4
Block_data:  Siva
Hash:  000006d60a97b31fd58e12d2645c0ce297437df2d90a6638c87ded0cca3f26b6
Nonce:  2453500

Add something to the chain: Shankaram

Previous_hash:  000006d60a97b31fd58e12d2645c0ce297437df2d90a6638c87ded0cca3f26b6
Block_data:  Shankaram
Hash:  000000537fca7e4a03c3ce8ebb8899f1db74bf551fc54fcac65d9ffc93d13895
Nonce:  98899

Add something to the chain: Poorana

Previous_hash:  000000537fca7e4a03c3ce8ebb8899f1db74bf551fc54fcac65d9ffc93d13895
Block_data:  Poorana
Hash:  0000031579db584f7d2331a95487b1641f0534e8e3a05829630ce975cc162bef
Nonce:  342626

Add something to the chain: Brahmam

Previous_hash:  0000031579db584f7d2331a95487b1641f053

In [ ]:
import hashlib,json
from datetime import datetime

class Block():
    def __init__(self,tstamp,voterInfo,previoushash=''):
        self.nonce = 0
        self.tstamp = tstamp
        self.voterInfo = voterInfo
        self.previoushash = previoushash
        self.hash = self.calcHash()
    
    def __str__(self):
        string  = "Nonce : " + str(self.nonce)+"\n"
        string += "voterInfo: " +str(self.voterInfo)+"\n"
        string += "Old hash: " +str(self.previoushash)+"\n"
        string += "New hash :" + str(self.hash)+"\n"
        return string 
    
    
    def calcHash(self):
        block_string = json.dumps({"Chain Nonce" : self.nonce,\
                                   "VotinggTimestamp" : str(self.tstamp),\
                                   "voterInfo" : self.voterInfo,\
                                   "previoushash":self.previoushash\
                                   }, sort_keys=True).encode()

        return hashlib.sha256(block_string).hexdigest()
        
    def mineBlock(self,difficulty):
        while(self.hash[:difficulty] != str('').zfill(difficulty)):
            self.nonce += 1
            self.hash = self.calcHash()


In [ ]:
class BlockChain():
    def __init__(self, difficulty):
        self.chain = [self.generateGenesisBlock(),]
        self.difficulty = difficulty

    def generateGenesisBlock(self):
        # Block (timeStamp, voterInfo, previousHash)
        return Block(None, None, None)

    def getLastBlock(self):
        return self.chain[-1]

    def addBlock(self,newBlock):
        newBlock.previoushash = self.getLastBlock().hash
        newBlock.mineBlock(self.difficulty)
        self.chain.append(newBlock)

    def isChainValid(self):
        for i in range(1,len(self.chain)):
            prevb = self.chain[i-1]
            currb = self.chain[i]
            if(currb.hash != currb.calcHash()):
                print("Invalid Block")
                return False
            if(currb.previoushash != prevb.hash):
                print("Invalid Chain")
                return False
        return True


In [ ]:
import datetime
print(datetime.datetime.now())

2021-12-10 10:54:52.205514


In [ ]:
bchain = BlockChain(3)
i=1

while i!="quit":

    age = int(input("Enter Your age: "))
   
    if age >= 18:
        print("You are eligible to vote! Cast your vote: ")
    else:
        print("Sorry, you are not eligible to vote")
        exit(1)
    
    # If eligible to vote ...
    name = str(input("Enter your name: "))
    
       
    vote = int(input(
                     "\nGiven below is the list of all sports:" + 
                     "\n 1. Kabaddi" + 
                     "\n 2. Kho-kho" +
                     "\n 3. Football" +
                     "\n 4. Swimming" + 
                     "\n 5. Archery" +
                     "\n 6. Gymnastics" + 
                     "\n 7. None of the above sports" +
                     "\nEnter your chosen option number: "
                    )
               )
    
    if vote == 1:
        chosen = "Kabaddi"
    elif vote ==2 :
        chosen = "Kho-kho"
    elif vote == 3:
        chosen = "Football"
    elif vote == 4:
        chosen = "Swimming"
    elif vote == 5:
        chosen = "Archery"
    elif vote == 6:
        chosen = "Gymnastics"
    else:
        chosen = "NOTA"
        exit(1)

    voteInfo = "Name: " + name +\
               "\nAge: " + str(age) +\
               "\nChosen sport: " + chosen

    bchain.addBlock(Block(datetime.now(),voteInfo))

    i = input("==============================================================="
              "\nType quit to close the system and get all voting details (or)" +
              "\nType cont for the next person to cast the vote " +
              "\n=============================================================="
             )

for b in bchain.chain:
    print(b)

Enter Your age: 21
You are eligible to vote! Cast your vote: 
Enter your name: Shiva Rupan S

Given below is the list of all sports:
 1. Kabaddi
 2. Kho-kho
 3. Football
 4. Swimming
 5. Archery
 6. Gymnastics
 7. None of the above sports
Enter your chosen option number: 2
Type quit to close the system and get all voting details (or)
Type cont for the next person to cast the vote 
==============================================================cont
Enter Your age: 20
You are eligible to vote! Cast your vote: 
Enter your name: Siva Shankar

Given below is the list of all sports:
 1. Kabaddi
 2. Kho-kho
 3. Football
 4. Swimming
 5. Archery
 6. Gymnastics
 7. None of the above sports
Enter your chosen option number: 4
Type quit to close the system and get all voting details (or)
Type cont for the next person to cast the vote 
==============================================================cont
Enter Your age: 24
You are eligible to vote! Cast your vote: 
Enter your name: Karthikeyan

Given b

# **Workout**

In [ ]:
h = hashlib.sha256()

In [ ]:
dir(h) # How to check class attributes????????????????

In [ ]:
h.update('SHANKARAM'.encode('utf-8'))

In [ ]:
h.hexdigest()

'bcedc77f1067d8e13334360b3de2da53fc0f0e6b4f84b2553c63615369dd3771'

In [ ]:
h.digest()

b'\xbc\xed\xc7\x7f\x10g\xd8\xe1346\x0b=\xe2\xdaS\xfc\x0f\x0ekO\x84\xb2U<caSi\xdd7q'

In [ ]:
2**256

115792089237316195423570985008687907853269984665640564039457584007913129639936

In [ ]:
int(h.hexdigest(), 16)

85454935161170513071890040701296470368552421092243685980428348606442123376497

In [ ]:
2**246 

113078212145816597093331040047546785012958969400039613319782796882727665664

In [ ]:
int(h.hexdigest(), 16) > 2**246


True

In [ ]:
i = 0 # Initial input 
h = hashlib.sha256() # Hash object
h.update('i'.encode('utf-8')) # Encoding '0' in utf-8 format

while (int(h.hexdigest(), 16) > 2**246):
  i = i+1 # updating the integer value
  h = hashlib.sha256()
  h.update(str(i).encode('utf-8'))

In [ ]:
i

286

In [ ]:
h.hexdigest()

'00328ce57bbc14b33bd6695bc8eb32cdf2fb5f3a7d89ec14a42825e15d39df60'

In [ ]:
print(int(h.hexdigest(), 16))
print(2**246)

89314784586040526161857519195072211904965977134279285863185261572107394912
113078212145816597093331040047546785012958969400039613319782796882727665664


In [ ]:
class Block():
  def __init__(self, data):
    self.data = data
    self.nonce = 0
    self.hash = hashlib.sha256()

  # Fix difficulty level, then obtain suitable number 'i'
  def mine(self, difficulty):
    
    h = hashlib.sha256(); h.update(str(self).encode('utf-8'))
    
    while int(h.hexdigest(), 16) > 2**(256-difficulty):
      self.nonce += 1
      h = hashlib.sha256()
      h.update(str(self).encode('utf-8'))

    self.hash = h
      

  def __str__(self):
    return "{}{}".format(self.data, self.nonce)
    

In [ ]:
obj = Block("Shankaram")
obj.mine(20)


In [ ]:
obj.hash.hexdigest()

'000003ad4f4ffaef077972d84abfcb359481e2b980e17431c1923b2cf087fa06'

In [ ]:
obj.nonce

1106070

In [ ]:
class what am i doing you are a leader you are to give hope to your team 

In [ ]:
class Chain():
  def __init__(self, difficulty):
    self.difficulty = difficulty